# Libraries

In [ ]:
!pip install noisereduce

In [ ]:

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy.fftpack import rfft, irfft, fftfreq, fft
from scipy.signal import butter, sosfilt, sosfreqz, filtfilt
from scipy.signal import hilbert

import librosa
import librosa.display
import IPython.display as ipd
import noisereduce as nr

from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans

import torch
import torchaudio

from math import ceil

import os
import json

import warnings
warnings.simplefilter('ignore')

# Base

In [ ]:
main_path = '/kaggle/input/birdclef-2022/'
print(f'[INFO] Archives:')
os.listdir(main_path)

In [ ]:
train_meta = pd.read_csv(main_path + 'train_metadata.csv')
test_data = pd.read_csv(main_path + 'test.csv')
ebird_data = pd.read_csv(main_path + 'eBird_Taxonomy_v2021.csv')
samp_subm = pd.read_csv(main_path + 'sample_submission.csv')

with open(main_path + 'scored_birds.json') as f:
    scored_birds = json.load(f)

train_meta_f = train_meta[train_meta.primary_label.isin(scored_birds)]
    
print('[INFO] Informations:')
print('\t-> Birds to be scored:', len(scored_birds))
print('\t-> Train:', train_meta.shape)
print('\t\t-> Class:', len(train_meta.primary_label.unique()))
print('\t-> Train Filtered:', train_meta_f.shape)
print('\t\t-> Class:', len(train_meta_f.primary_label.unique()))
print('\t-> Test:',test_data.shape)

In [ ]:
train_meta_f.head()

In [ ]:
test_data.head()

# Graphics

In [ ]:
train_meta_f['count'] = 1
g_00 = train_meta_f.groupby(['primary_label'])['count'].count()\
                   .reset_index()\
                   .sort_values('count',ascending=False)

g_01 = train_meta_f.groupby(['secondary_labels'])['count'].count()\
                   .reset_index()\
                   .sort_values('count',ascending=False)
                                           
fig, (ax1,ax2) = plt.subplots(2, 1, figsize=(8,10))
fig.suptitle('Graphics')
ax1 = sns.barplot(x ='count',y='primary_label', data = g_00.head(10), ax=ax1)
ax1.set_title('Primary Label')
ax1.set(xlabel ='Count', ylabel='Label')

ax2 = sns.barplot(x ='count',y='secondary_labels', data = g_01.head(10), ax=ax2)
ax2.set_title('Secondary Label')
ax2.set(xlabel ='Count', ylabel='Label')

plt.show()

In [ ]:
label = 'skylar'

df_00 = train_meta_f[train_meta_f.primary_label == label][['primary_label','latitude','longitude']]
fig = px.density_mapbox(data_frame=df_00, 
                        lat='latitude', 
                        lon='longitude',
                        radius=5,
                        center = dict(lat=10,lon=-2),
                        zoom = 0.9,
                        mapbox_style='stamen-terrain',
                        title = f'Density map: {label} ')
fig.show()

In [ ]:
fig = px.scatter_geo(
    train_meta_f,
    lat="latitude",
    lon="longitude",
    color = 'primary_label',
    width=900,
    height=500,
    title="Training Data (Filtered)",
    hover_name= 'primary_label'
)
fig.show()

# Graphics (K means)

In [ ]:
def KMeans_Clusters(data, n = 5):
    
    sum_of_squared_distances = []
    models = []
    K_groups = range(2, n + 1 , 1)
    
    for k in K_groups:
        kmeans = KMeans(n_clusters = k,
                       random_state = 42)
        kmeans.fit(data)
        
        sum_of_squared_distances.append(kmeans.inertia_)
        models.append(kmeans)
    
    x1, y1 = 2, sum_of_squared_distances[0]
    x2, y2 = n, sum_of_squared_distances[-1]
    
    x_point = [x1, x2]
    y_point = [y1, y2]
    
    distances = []
    
    for i in range(len(sum_of_squared_distances)):
        x0 = i+2
        y0 = sum_of_squared_distances[i]
        a = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
        b = np.sqrt((y2 - y1)**2 + (x2 - x1)**2)
        distances.append(a/b)
    
    i_max = distances.index(max(distances))
    k_best = i_max + 2
    model_best = models[i_max]
    print('[INFO] Best K = ', k_best)

    plt.plot(K_groups, sum_of_squared_distances, 'rx-')
    plt.plot(x_point, y_point, 'b--')
    plt.xlabel('k')
    plt.ylabel('Sum_of_squared_distances')
    plt.title('Elbow Method For Optimal k')
    plt.show()
    
    return model_best
        
kmeans = KMeans_Clusters(data = train_meta_f[['latitude','longitude']],
                         n = 20)
train_meta_f['Group'] = kmeans.labels_

In [ ]:
fig = px.scatter_geo(
    train_meta_f,
    lat="latitude",
    lon="longitude",
    color="Group",
    width=900,
    height=500,
    title="Training Data (Filtered)",
    hover_name= 'primary_label'
)
fig.show()

# Signal

In [ ]:
def Get_Params(y_t, sr):
    N = len(y_t)                                      
    T = 1.0 / sr                                      
    x_t = np.linspace(0.0, N*T, N)                    
    x_f = np.linspace(0.0, 1.0/(2.0*T), int(N/2))      
    return N, T, x_t, x_f

def Remove_Continuos(y_t):
    mean_y = np.mean(y_t)
    y_ = y_t - mean_y
    return y_

def Normalization(y_t):
    max_y = np.max(y_t)
    y_ = y_t/max_y
    return y_

def Noice_Reduce(y_t, sr, n_fft = 1024, win_length = 512, use_tqdm = False):
    y_ = nr.reduce_noise(y = y_t,
                        sr = sr,
                        n_fft = n_fft,
                        win_length = win_length,
                        use_tqdm = use_tqdm,
                        n_jobs = 2)
    y_ = np.array(y_)
    return y_

def Limit(y_t, sr, x_f, N , taxa = 1.5):
    y_f = fft(y_t)
    y_f = 2.0/N * np.abs(y_f[:N//2])
    mean_yf = np.mean(y_f)
    std_yf = np.std(y_f)
    distances = np.power(y_f - mean_yf, 2) 
    index = distances.argmin()

    if x_f[index]*taxa >= np.max(x_f):
        freq_cut = np.max(x_f)*0.95
    else:
        freq_cut = x_f[index]*taxa 
        
    return freq_cut

def Band_Pass_Filter(y_t, sr, lowcut, highcut, order=5):
    nyq = 0.5 * sr
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='bandpass', analog=False, output='ba')
    y_ = filtfilt(b, a, y_t)
#     sos = butter(order, normal_cutoff , analog=False, btype='lowpass', output='sos')
#     y = sosfilt(sos, data)
    return y_

def Envelope(y_t):
    analytical_signal = hilbert(y_t)
    y_ = np.abs(analytical_signal)
    hilbert_fft = fft(y_)
    return y_, hilbert_fft


def Complete_Treatment(y_t, sr, n_fft = 1024, win_length = 512, use_tqdm = False, plot=True, nome=None):
    N, T, x_t, x_f = Get_Params(y_t, sr)
    y_f_0 = fft(y_t)
    
    y_1 = Remove_Continuos(y_t)
    y_f_1 = fft(y_1)
    
    y_2 = Normalization(y_1)
    y_f_2 = fft(y_2)   

    y_3 = Noice_Reduce(y_2, 
                        sr, 
                        n_fft = n_fft, 
                        win_length = win_length, 
                        use_tqdm = use_tqdm)
    y_f_3 = fft(y_3)
    
    freq_cut = Limit(y_3, sr, x_f, N, taxa = 1.2)
    y_4 = Band_Pass_Filter(y_3, sr, lowcut = 100, highcut = freq_cut, order = 5)
    y_f_4 = fft(y_4)
    
    y_5, y_f_5 = Envelope(y_4)
    
    
    if plot:
        fig, ax = plt.subplots(figsize =(15,12), nrows = 6, ncols = 2)
        if nome == None:
            fig.suptitle('Signal Treatment')
        else:
            fig.suptitle(f'Signal Treatment - bird: {nome}')
            
        ax[0, 0].plot(x_t, y_t)
        ax[0, 0].set_title('Raw Data')
        ax[0, 0].set_xlabel('t')   
        ax[0, 1].plot(x_f, 2.0/N * np.abs(y_f_0[:N//2]))
        ax[0, 1].set_title('Frequency')
        ax[0, 1].set_xlabel('Hz')
        
        ax[1, 0].plot(x_t, y_1)
        ax[1, 0].set_title('Data Without Continuous Component')
        ax[1, 0].set_xlabel('t')   
        ax[1, 1].plot(x_f, 2.0/N * np.abs(y_f_1[:N//2]))
        ax[1, 1].set_title('Frequency')
        ax[1, 1].set_xlabel('Hz')  
        
        ax[2, 0].plot(x_t, y_2)
        ax[2, 0].set_title('Normalized Data')
        ax[2, 0].set_xlabel('t')   
        ax[2, 1].plot(x_f, 2.0/N * np.abs(y_f_2[:N//2]))
        ax[2, 1].set_title('Frequency')
        ax[2, 1].set_xlabel('Hz')  
        
        ax[3, 0].plot(x_t, y_3)
        ax[3, 0].set_title('Data With Noise Reduce')
        ax[3, 0].set_xlabel('t')   
        ax[3, 1].plot(x_f, 2.0/N * np.abs(y_f_3[:N//2]))
        ax[3, 1].set_title('Frequency')
        ax[3, 1].set_xlabel('Hz')  
        
        ax[4, 0].plot(x_t, y_4)
        ax[4, 0].set_title('Filtered Data')
        ax[4, 0].set_xlabel('t')   
        ax[4, 1].plot(x_f, 2.0/N * np.abs(y_f_4[:N//2]))
        ax[4, 1].set_title('Frequency')
        ax[4, 1].set_xlabel('Hz')
        
        # Envelope
        ax[5, 0].plot(x_t, y_5)
        ax[5, 0].set_title('Envelope')
        ax[5, 0].set_xlabel('t')   
        ax[5, 1].plot(x_f, 2.0/N * np.abs(y_f_5[:N//2]))
        ax[5, 1].set_title('Frequency')
        ax[5, 1].set_xlabel('Hz')
        
        plt.subplots_adjust(wspace = 0.2, hspace = 0.8)
        plt.show()
        
    return y_5, y_4, x_t

# Criação do Espectograma

In [ ]:
# Cria o Espectograma
def Criate_Spectrograms(file_name, 
                         frame_size = 10,
                         frame_step = 2, 
                         channel = 0,
                         signal_treatment = True, 
                         n_fft = 1800, 
                         win_length = 512,
                         hilbert = True,
                         device = 'cpu'):
#     y_t, sr = torchaudio.load(file_name)
    y_t, sr = librosa.load(file_name) 
    transform = torchaudio.transforms.Spectrogram(n_fft = n_fft, win_length = win_length).to(device)
    if signal_treatment :
        y_hilbert, y_, _ = Complete_Treatment(y_t, sr, win_length = transform.win_length, plot=False)
        if hilbert:
            y_t = torch.from_numpy(np.array(y_hilbert))
        else:
            y_t = torch.from_numpy(np.array(y_))
#     print(y_t.size())
    step = int(frame_step * sr)
    size = int(frame_size * sr)
    spectrograms = []
    
    for i in range(ceil((y_t.size()[-1] - size) / step)):
        begin = i * step
        frame = y_t[begin:begin + size]
        if len(frame) < size:
            if i == 0:
                rep = round(float(size) / len(frame))
                frame = frame.repeat(int(rep))
            elif len(frame) < (size * 0.33):
                continue
            else:
                frame = y_t[-size:]
        sg = transform(frame.to(device))
        spectrograms.append(np.nan_to_num(torch.log(sg).numpy()))
        # spectrograms.append(np.nan_to_num(sg.numpy()))
    return spectrograms  

# Test

In [ ]:
for i in range(5):
    path_audio = main_path + 'train_audio/'
    n = np.random.randint(0, len(train_meta_f))
    audio = train_meta_f.filename.values[n]
    bird = train_meta_f.primary_label.values[n]

    y_t, sr = librosa.load(path_audio+audio) 


    y_trat, _, x_t = Complete_Treatment(y_t, sr, plot=True, nome=bird)

    sgs_h = Criate_Spectrograms(path_audio+audio,
                              frame_size = int(x_t[-1]),
                              frame_step = int(x_t[-1]))
    sgs = Criate_Spectrograms(path_audio+audio,
                              frame_size = int(x_t[-1]),
                              frame_step = int(x_t[-1]),
                              hilbert = False)
    plt.figure(figsize=(15,5))
    plt.imshow(sgs_h[0], interpolation='nearest', aspect='auto')
    plt.title(f'Spectrogram (with Hilbert) - bird: {bird}')
    plt.show()
        
    plt.figure(figsize=(15,5))        
    plt.imshow(sgs[0], interpolation='nearest', aspect='auto')
    plt.title(f'Spectrogram (without Hilbert) - bird: {bird}')
    plt.show()
